# Agent Tools

In [1]:
import os
from agents import Agent, Runner, WebSearchTool, function_tool
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Hosted Tools

These are tools provided to us by OpenAI. They are plug and play which means they don't require us to write out functions or do much configuration.

### Custom Tools

These are tools we build ourselves and provide our agents access to use them. Remember that all they are, are python functions that do a specific task (eg. call an API, get data from a database, perform a calculation/logic, etc.)

### Agents as Tools

We can provide our agents access to other agents as tools. This differs slightly from handoffs which we will have a look at later.

# Agent Handoffs

This is a different concept to agents as tools. Think of agents as tools as a way to supplement your main agent. Your main agent is able to make use of other tools and use the output that it gets back but the conversation with the user remains with the main agent.

In handoffs, you handoff the conversation to another agent (lets say a sales agent).

**Why might we do this?**
The use cases might be so niche that both these options will seem similar in terms of the actual output we get, the key difference is in the system instructions given to each agent. Each agent is going to have its own set of instructions and knowledge it has access to (as well as tone and style of writting for example) and this might be the deciding factor of wanting to use agents as tools or handoffs.